In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# The contracting path
model_seq = nn.Sequential()

# First encoding layer
model_seq.append(nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())
model_seq.append(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())

# First pooling layer
model_seq.append(nn.MaxPool2d(kernel_size=2, stride=2))

# Second encoding layer
model_seq.append(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())
model_seq.append(nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())

# Second pooling layer
model_seq.append(nn.MaxPool2d(kernel_size=2, stride=2))

# Third encoding layer
model_seq.append(nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())
model_seq.append(nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())

# Third pooling layer
model_seq.append(nn.MaxPool2d(kernel_size=2, stride=2))

# Fourth encoding layer
model_seq.append(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())
model_seq.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())

# Fourth pooling layer
model_seq.append(nn.MaxPool2d(kernel_size=2, stride=2))

# Bottleneck
model_seq.append(nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())
model_seq.append(nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())


# Upsampling 
model_seq.append(nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=2, stride=2))
# After concatenating skip connection, apply two conv layers
model_seq.append(nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())
model_seq.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())

# Second upsampling
model_seq.append(nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2))
# After concatenating skip connection, apply two conv layers
model_seq.append(nn.Conv2d(512, 256, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())
model_seq.append(nn.Conv2d(256, 256, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())

# Third upsampling
model_seq.append(nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2))
# After concatenating skip connection, apply two conv layers
model_seq.append(nn.Conv2d(256, 128, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())
model_seq.append(nn.Conv2d(128, 128, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())

# Fourth upsampling 
model_seq.append(nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2))
# After concatenating skip connection, apply two conv layers
model_seq.append(nn.Conv2d(128, 64, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())
model_seq.append(nn.Conv2d(64, 64, kernel_size=3, padding=1))
model_seq.append(nn.ReLU())

# Final output layer
model_seq.append(nn.Conv2d(64, 1, kernel_size=1))

In [32]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = F.interpolate(x, size=skip_connection.shape[2:], mode='bilinear', align_corners=True)

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

def test():
    x = torch.randn((3, 1, 161, 161))
    model = UNET(in_channels=1, out_channels=1)
    preds = model(x)
    assert preds.shape == x.shape